In [2]:
!pip install openai pandas PyPDF2 tqdm

   ---------------------------------------- 0.0/606.1 kB ? eta -:--:--
   --------------------------------------- 606.1/606.1 kB 11.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 15.7 MB/s eta 0:00:00


해당 코드를 진행하기 위해 OpenAI API 가입 및 결제가 선행되어야 합니다.

OpenAI API 가이드에 따라서 API_KEY를 발급 받으신 후 활용하세요.

활용 가능한 모델은 [https://platform.openai.com/docs/models](https://platform.openai.com/docs/models) 에서 확인할 수 있습니다.

** API key를 발급받으신 후 'Your-OpenAI-Key' 문자열 검색 및 해당 자리에 발급받은 OpenAI-key 키를 넣어주세요.

In [ ]:
import openai
import pandas as pd
import PyPDF2
import os
import time
import logging
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("pdf_summarizer.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger()

class PDFSummarizer:
    def __init__(self, api_key=None, model="o3-mini-2025-01-31"): # 사용하고자 하는 model 명시시 
        """초기화 함수"""
        self.api_key = api_key or os.environ.get("OPENAI_API_KEY")
        if not self.api_key:
            raise ValueError("API 키가 필요합니다. 환경 변수 OPENAI_API_KEY를 설정하거나 직접 전달해주세요.")
        
        openai.api_key = self.api_key
        self.model = model
        logger.info(f"PDFSummarizer 초기화 완료 (모델: {model})")
    
    def extract_text_from_pdf(self, file_path):
        """PDF 파일에서 텍스트를 추출하는 함수"""
        if not os.path.exists(file_path):
            logger.error(f"파일을 찾을 수 없습니다: {file_path}")
            return ""
        
        try:
            text = ""
            with open(file_path, "rb") as file:
                pdf_reader = PyPDF2.PdfReader(file)
                for page_num, page in enumerate(pdf_reader.pages):
                    page_text = page.extract_text() or ""
                    text += page_text
                    
            # 텍스트가 너무 짧으면 경고
            if len(text.strip()) < 100:
                logger.warning(f"추출된 텍스트가 매우 짧습니다: {file_path}")
                
            return text
            
        except Exception as e:
            logger.error(f"PDF 텍스트 추출 오류 ({file_path}): {str(e)}")
            return ""
    
    def summarize_text(self, text, max_tokens=300, temperature=0.5, prompt_template=None):
        """텍스트 요약 함수 (ChatGPT API 활용)"""
        if not text.strip():
            logger.warning("요약할 텍스트가 없습니다.")
            return "요약할 텍스트가 없습니다."
        
        # 기본 프롬프트 템플릿
        default_template = "다음 의학 텍스트를 요약해주세요. 주요 개념, 중요 용어, 핵심 내용을 포함하세요:\n\n{text}\n\n요약:"
        prompt = (prompt_template or default_template).format(text=text)
        
        # API 호출 시도 (오류 시 최대 3번 재시도)
        max_retries = 3
        for attempt in range(max_retries):
            try:
                response = openai.chat.completions.create(
                    model=self.model,
                    messages=[{"role": "user", "content": prompt}],
                    max_tokens=max_tokens,
                    temperature=temperature
                )
                return response.choices[0].message.content.strip()
            
            except Exception as e:
                logger.warning(f"API 호출 오류 (시도 {attempt+1}/{max_retries}): {str(e)}")
                if attempt < max_retries - 1:
                    time.sleep(2 ** attempt)  # 지수 백오프
                else:
                    logger.error(f"API 호출 실패: {str(e)}")
                    return f"요약 실패: {str(e)[:100]}..."
    
    def process_single_pdf(self, file_path, max_tokens=300, temperature=0.5, prompt_template=None):
        """단일 PDF 파일 처리"""
        file_name = os.path.basename(file_path)
        logger.info(f"파일 처리 시작: {file_name}")
        
        text = self.extract_text_from_pdf(file_path)
        
        # 텍스트가 너무 길면 청크로 나누기
        if len(text) > 10000:
            logger.info(f"텍스트가 길어서 청크로 나누어 요약합니다: {file_name}")
            chunks = self._split_text(text)
            chunk_summaries = []
            
            for i, chunk in enumerate(chunks):
                logger.info(f"청크 {i+1}/{len(chunks)} 요약 중...")
                chunk_summary = self.summarize_text(chunk, max_tokens=150, temperature=temperature)
                chunk_summaries.append(chunk_summary)
            
            # 청크 요약들을 다시 한번 요약
            combined_summary = "\n\n".join(chunk_summaries)
            final_summary = self.summarize_text(
                combined_summary, 
                max_tokens=max_tokens,
                temperature=temperature,
                prompt_template="다음은 텍스트의 부분 요약들입니다. 이것들을 하나의 일관된 요약으로 통합해주세요:\n\n{text}\n\n최종 요약:"
            )
        else:
            final_summary = self.summarize_text(
                text, 
                max_tokens=max_tokens,
                temperature=temperature,
                prompt_template=prompt_template
            )
        
        logger.info(f"파일 처리 완료: {file_name}")
        return {
            "file_path": file_path,
            "file_name": file_name,
            "summary": final_summary,
            "text_length": len(text)
        }
    
    def _split_text(self, text, max_chunk_size=5000, overlap=200):
        """긴 텍스트를 청크로 나누는 함수"""
        chunks = []
        start = 0
        
        while start < len(text):
            end = min(start + max_chunk_size, len(text))
            
            # 문장 중간에 잘리지 않도록 조정
            if end < len(text):
                # 마침표, 줄바꿈 등으로 끝나는 위치 찾기
                for sep in ['. ', '.\n', '\n\n', '\n', '. ', '? ', '! ']:
                    pos = text.rfind(sep, start, end)
                    if pos != -1:
                        end = pos + len(sep)
                        break
            
            chunks.append(text[start:end])
            start = end - overlap  # 겹치는 부분 유지
            
        return chunks
    
    def summarize_pdfs_to_csv(self, pdf_files, output_csv, max_workers=4, max_tokens=300, temperature=0.5, prompt_template=None):
        """여러 PDF 파일을 처리하고 요약 결과를 저장하는 함수"""
        if not pdf_files:
            logger.warning("처리할 PDF 파일이 없습니다.")
            return
        
        results = []
        start_time = time.time()
        
        # 병렬 처리
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            futures = {
                executor.submit(
                    self.process_single_pdf, 
                    file_path, 
                    max_tokens, 
                    temperature,
                    prompt_template
                ): file_path for file_path in pdf_files
            }
            
            # tqdm으로 진행률 표시
            for future in tqdm(futures, desc="PDF 요약 진행 중"):
                try:
                    result = future.result()
                    results.append(result)
                except Exception as e:
                    file_path = futures[future]
                    logger.error(f"파일 처리 실패 ({file_path}): {str(e)}")
                    results.append({
                        "file_path": file_path,
                        "file_name": os.path.basename(file_path),
                        "summary": f"처리 실패: {str(e)[:100]}...",
                        "text_length": 0
                    })
        
        # 결과 저장
        df = pd.DataFrame(results)
        
        # 디렉토리 확인 및 생성
        output_dir = os.path.dirname(output_csv)
        if output_dir and not os.path.exists(output_dir):
            os.makedirs(output_dir)
            
        # CSV 저장
        df.to_csv(output_csv, index=False, encoding='utf-8-sig')
        
        elapsed_time = time.time() - start_time
        logger.info(f"요약 작업 완료: {len(results)}개 파일 처리됨 (소요시간: {elapsed_time:.2f}초)")
        logger.info(f"결과가 {output_csv}에 저장되었습니다.")
        
        return df

# 사용 예시
if __name__ == "__main__":
    # API 키 설정 (환경 변수에서 가져오거나 직접 지정)
    API_KEY = "Your-OpenAI-Key"
    
    # PDF 파일 경로 설정
    # pdf_dir = "data/medical_papers/"
    pdf_dir = r"C:\Users\inhag\Desktop\Paper\yes"
    output_csv = "results/summarized_medical_papers.csv"
    
    # 디렉토리에서 모든 PDF 파일 찾기
    pdf_files = []
    if os.path.exists(pdf_dir):
        pdf_files = [os.path.join(pdf_dir, f) for f in os.listdir(pdf_dir) if f.lower().endswith('.pdf')]
    
    # 없으면 직접 경로 지정
    if not pdf_files:
        pdf_files = [
            "path/to/your/file1.pdf", 
            "path/to/your/file2.pdf"
        ]
    
    # 요약 객체 생성 및 실행
    summarizer = PDFSummarizer(api_key=API_KEY)
    
    # 의학 논문 맞춤형 프롬프트 템플릿
    medical_prompt = """
    다음 의학 텍스트를 전문적이고 체계적으로 요약해주세요:
    1. 주요 질환이나 상태
    2. 연구 방법 및 결과
    3. 중요한 의학 용어와 그 정의
    4. 임상적 의의
    
    원문:
    {text}
    
    요약:
    """
    
    # 실행
    results_df = summarizer.summarize_pdfs_to_csv(
        pdf_files=pdf_files,
        output_csv=output_csv,
        max_workers=4,  # 병렬 처리 워커 수
        max_tokens=400,  # 요약 길이
        temperature=0.3,  # 더 일관된 결과를 위해 낮은 온도
        prompt_template=medical_prompt
    )
    
    # 결과 미리보기
    print("\n요약 결과 미리보기:")
    print(results_df[["file_name", "summary"]].head())

2025-03-25 07:15:06,549 - INFO - PDFSummarizer 초기화 완료 (모델: o3-mini-2025-01-31)
2025-03-25 07:15:06,550 - INFO - 파일 처리 시작: diagnostics-12-02679.pdf
2025-03-25 07:15:06,552 - INFO - 파일 처리 시작: ryai.2020200198.pdf
2025-03-25 07:15:06,554 - INFO - 파일 처리 시작: s12891-022-05818-4.pdf
PDF 요약 진행 중:   0%|          | 0/3 [00:00<?, ?it/s]2025-03-25 07:15:06,933 - INFO - 텍스트가 길어서 청크로 나누어 요약합니다: diagnostics-12-02679.pdf
2025-03-25 07:15:07,424 - INFO - 텍스트가 길어서 청크로 나누어 요약합니다: ryai.2020200198.pdf
2025-03-25 07:15:07,786 - INFO - 텍스트가 길어서 청크로 나누어 요약합니다: s12891-022-05818-4.pdf
